The overall map of the statement generation and prompts.


In [0]:

# stages model:
#
# this models the overall structure of the statement flows of a personal statement.
# The next stage(s) possible are labeled as "links" from a given template.
# We store the visited links in a progress list as we traverse the flow
# and check to make sure we're not cycling back over a stage we've already visited.
#
stages = {
    "intro": {
        "id": 0,
        # any response links to next stage, "name"
        "links": "name",
        "template": """...Let's begin with paragraph one, where you will introduce yourself to the
        judge reviewing your records.""",
        "input_type": "short-input"
    },
    "name": {
        "id": 100,
        # any response links to next stage, "age"
        "links": "age",
        "template": "Please enter your full name as you'd like to see it on your personal statement: ",
        "input_type": "short-input",
    },
    "age": {
        "id": 102,
        # input_type of "short-input" takes free text and progresses to the next link
        # regardless of response
        "links": "working",
        "template": " Please enter your age in years: ",
        "input_type": "short-input",
    },
    "working": {
        "id": 120,
        # input_type of yes-no can link to one or more next stages based on response
        # so links is a key-value pair of response to next stage
        "links": {
            # affirmative responses link to "industry-current" stage
            "yes": "industry-current",
            "yes, inconsistently": "industry-current",

            # specific negative response link to their respective stage
            "no, but I was until recently": "industry-past",
            "no": "industry-past-explanation",
        },
        "template": """Are you currently working? Choose the closest answer: 'Yes', 'Yes,
inconsistently', 'No, but I was until recently', 'No'""",
        "input_type": "yes-no",
    },
    "industry-current": {
        "id": 121,
        "links": "industry-current-duration",
        "template": "In what industry are you working?",
        "input_type": "short-input",
    },
    "industry-current-duration": {
        "id": 122,
        # jump to the end
        "links": "END",
        "template": """How long have you been working in this industry? Complete this sentence for
        your personal statement: I have been working in this industry for ____""",
        "input_type": "short-input",
    },
    "industry-past": {
        "id": 131,
        "links": "industry-past-duration",
        "template": "In what industry were you working?",
        "input_type": "short-input",
    },
    "industry-past-duration": {
        "id": 132,
        "links": "END",
        "template": """How long have you been working in this industry? Complete this sentence for
        your personal statement: I have been working in this industry for ____""",
        "input_type": "short-input",
    },
    "industry-past-explanation": {
        "id": 133,
        "links": {"yes": "industry-past-explanation-short-input", "no": "END"},
        "template": """Would you like to add a sentence explaining your work situation to the judge? We recommend this
    if you think it adds context, particularly if the reason that you are having trouble finding work
is related to your conviction.""",
        "input_type": "yes-no",
    },
    "industry-past-explanation-short-input": {
        "id": 134,
        "links": "END",
        "template": "Please enter a short explanation of why you're having trouble finding work.",
        "input_type": "short-input",
    },
    "END": {"title": "END", "id": -1, "links": [], "template": "You have reached the end of this form."},
}

General functions for ease of use and readability. For user testing, just run this next block so that the functions are defined for later, but don't worry too much about it. Click the grey play button on the block below.

In [0]:
from datetime import date

NEWLINE = "\n"
YES_NO = ["Yes", "No"]


def my_function():
  print("this is mine")

def query_user(prompt, responses=[]):
  print(prompt)                                                                                                                                                              
  response_dict = {}
  count = 1
  for response in responses:                                                                                                                                                      
    response_dict[count] = response
    print(count, " - ", response)
    count += 1                                                                                                                                                          
  
  if responses:
    while True:                                                                                                                                                                     
      inp = input()
      if inp.isdigit() and int(inp) in response_dict:
        break
      print("That is not an option. Try again.")
    return response_dict[int(inp)]

  inp = input()
  return inp.strip()


def add_context():
  wants_context = query_user("Would you like to add a sentence of context to your prior response?", YES_NO)
  if wants_context == "Yes":
    return query_user("Okay. Please add your sentence here: ")
  return ""

def print_personal_statement(statement):
  # print the completed statement
  for p in personal_statement:
    print(p)
    print(NEWLINE)

# exceptions
class NextStageLinkTypeError(Exception):
    """ next linked stage is incorrect type """

class ResponseNotHandledError(Exception):
    """ the response is not handled """

# response handlers
def noop(links, response):
    """ no operation placeholder """
    return links


def shortinput(links, response):
    """ return the next linked stage """
    if not isinstance(links, (str)):
        raise NextStageLinkTypeError("oops something broke")
    return links


def yesno(links, response) -> int:
    """
    example response parser
    params:
        links - list; a list of Stage Id integers of next possible links
        response - str; the raw input response data
    """
    response_data = response.lower()

    if not isinstance(links, (dict)):
        raise NextStageLinkTypeError("oops something broke")

    if response_data not in set(links.keys()):
        raise ResponseNotHandled("could not understand the response")

    return links[response_data]


def response_parser(stage, response_data):
    """
    apply the parser correspnoding to stage["input_type"]
    with the given response data.

    returns the next link(s) for the given response

    params:
        stage - dict; the current stage data
        response_data - string; raw response input from prompt
    returns:
        int - the stage id next
    """
    fn = None
    input_type = stage.get("input_type", None)

    if input_type is None:
        fn = noop
    elif input_type == "yes-no":
        fn = yesno
    elif input_type == "short-input":
        fn = shortinput
    else:
        fn = noop

    return fn(stage["links"], response_data)


In [0]:
def main(stage_model, stage="0", progress=[]):
    """ main """
    # are we at the final stage?
    if stage == "END":
        # print last template, record progress, and exit
        print(stage_model[stage]["template"])
        progress.append({"id": stage})
        return progress

    # have we been here already?
    # if so, go to the end
    if stage in set(x["id"] for x in progress):
        return main(stage_model, stage="-1", progress=progress)

    # run the current stage and progress
    current_stage = stage_model[stage]
    response = query_user(current_stage["template"])

    # do some logic or validation here on response data ...
    # based on the current_stage "input-type"
    next_stage = response_parser(current_stage, response)

    # record progress
    progress.append({"id": current_stage["id"], "response": response})

    return main(stage_model, stage=next_stage, progress=progress)





Paragraph 1: The following block outputs paragraph 1 of the statement. To test the user interactions for paragraph 1, click the play button by the block below.

In [0]:
def build_paragraph_one():
  personal_statement = []
  pp0 = ""
  pp0 += str(date.today()) + NEWLINE + NEWLINE
  pp0 += "To Whom It May Concern,"

  personal_statement.append(pp0)

  print("Hello. We'd like to help you write a better personal statement that will improve your chances of getting the record clearance" + NEWLINE +
        "that you asked for. To do that, we're going to help you write your personal statement sentence by sentence. This is your " + NEWLINE +
        "chance to explain yourself to the judge so they understand you and your situation better. It's important to put your best " + NEWLINE +
        "foot forward. Remember, the majority of requests for getting records cleared are approved, so it doesn't need to be perfect. " + NEWLINE +
        "Just show the judge that you care, that you are putting in effort to clear your record, and why it is important to you." + NEWLINE +
        NEWLINE +
        "Let's begin with paragraph one, where you will introduce yourself to the judge reviewing your records. ")

  pp1 = ""
  name = query_user("Please enter your full name as you'd like to see it on your personal statement: ")
  age = query_user("Please enter your age in years: ")
  pp1 += "My name is " + str(name) + ", and I am " + str(age) + " years old. "

  is_close_to_family = query_user("Do you have family that you talk to? ", YES_NO)
  if is_close_to_family == "Yes":
    family_members = query_user("Please complete this sentence for your personal statement:\nI have a close family, including **YOUR RESPONSE* .")
    pp1 += "I have a close family, including " + str(family_members) + ". "

  is_working = query_user("Are you currently working? Choose the closest answer: ", ["Yes", "Yes, inconsistently", "No, but I was until recently", "No"])
  if is_working == "Yes" or is_working == "Yes, inconsistently":
    industry = query_user("In what industry are you working? ")
    duration = query_user("How long have you been working in this industry? Complete this sentence for your personal statement:" + NEWLINE +
                          "I have been working in " +  str(industry) + " for *YOUR RESPONSE* .")
    pp1 += "I have been working in " +  str(industry) + " for " + str(duration) + ". "

  elif is_working == "No, but I was until recently":
    industry = query_user("In what industry were you working? ")
    duration = query_user("How long were you been working in this industry? Complete this sentence for your personal statement:" + NEWLINE +
                          "Until recently I worked in " +  str(industry) + " for *YOUR RESPONSE* .")
    pp1 += "Until recently I worked in " + str(industry) + " for " + str(duration) + ". "

  elif is_working == "No":
    add_response = query_user("Would you like to add a sentence explaining your work situation to the judge? We recommend this " + NEWLINE +
                              "if you think it adds context, particularly if the reason that you are having trouble finding " + NEWLINE +
                              "work is related to your conviction.", YES_NO)
    if add_response == "Yes":
      pp1 += query_user("Okay. Please add your sentence here: ")
    else:
      print("Okay. ")

  print("Now, for the most sincere part of the first paragraph. In the next three sentences you are going to explain to the judge why " + NEWLINE +
        "you want your record cleared. The structure goes like this: " + NEWLINE +
        "Sentence 1 explains what is hard or difficult about your life right now. For example, 'I want my own apartment, but renters " + NEWLINE +
        "often turn me down because of my record.'" + NEWLINE +
        "Sentence 2 explains what challenges you face because of Sentence 1. For example, 'I am living with friends and family when " + NEWLINE +
        "I can.'" + NEWLINE +
        "Sentence 3 states that you would like to clear your record and explains how clearing your record would help with this goal." + NEWLINE +
        "For example, 'I would like to clear my record so that I can get approved to rent my own apartment.'" + NEWLINE + NEWLINE +
        "You can skip any one sentence you like, but please try to enter at least two of them, and make sure your responses make sense together")
  pp1 += query_user("Please enter your Sentence 1: ")
  pp1 += query_user("Please enter your Sentence 2: ")
  pp1 += query_user("Please enter your Sentence 3: ")

  personal_statement.append(pp1)
  print("We've finished the first of your five paragraph statement. Moving on to paragraph 2." + NEWLINE) 

  print_personal_statement(personal_statement)

build_paragraph_one()



Hello. We'd like to help you write a better personal statement that will improve your chances of getting the record clearance
that you asked for. To do that, we're going to help you write your personal statement sentence by sentence. This is your 
chance to explain yourself to the judge so they understand you and your situation better. It's important to put your best 
foot forward. Remember, the majority of requests for getting records cleared are approved, so it doesn't need to be perfect. 
Just show the judge that you care, that you are putting in effort to clear your record, and why it is important to you.

Let's begin with paragraph one, where you will introduce yourself to the judge reviewing your records. 
Please enter your full name as you'd like to see it on your personal statement: 


Paragraph 2: The following block outputs paragraph 2 of the statement. To test the user interactions for paragraph 2, click the play button by the block below.

In [0]:
def build_paragraph_two():
  personal_statement = []
  print("This paragraph explains the changes in your life since your conviction so the judge can see how you've changed.")
  pp2 = ""

  something_has_changed = query_user("We would like to show the judge that you are actively working to change your situation. " + NEWLINE +
                                     "Please fill in this sentence for your personal statement: " + NEWLINE +
                                     "Since my last conviction, *YOUR RESPONSE*. ")
  pp2 += "Since my last conviction, " + something_has_changed + ". "
  pp2 += add_context()

  while (True):
    more = query_user("Has anything else changed in your life changed that you would like to share?", YES_NO)
    if more == "Yes":
      change = query_user("Please complete this sentence with your change:\n Also, *YOUR RESPONSE*.")
      pp2 += "Also, " + change + ". "
      pp2 += add_context() + " "
    else:
      break

  why_it_matters = query_user("Now, to tie paragraph 2 together, we'll relate the changes that you listed above to the reason" + NEWLINE +
                              "that you want to clear your record. Think over all the changes you've told the judge above. " + NEWLINE +
                              "Please write a sentence explaining how a few of your changes could be better for you if " + NEWLINE +
                              "your record were cleared.")
  pp2 += why_it_matters + " "

  personal_statement.append(pp2)
  print_personal_statement(personal_statement)

build_paragraph_two()

Using the stages model: Run through the stages, following links to sub stages based on the corresponding response. Returns a list of progression through the form with all reponses attached for formatting/parsing into final shape.

In [0]:
  prog = main(stages, stage="intro", progress=[])
  print(f"====== Final collected responses =============")
  print(prog)